# Putting it all together

Let us look at a worked example combining much of what we've seen so far.

Suppose we have developed a plasma diagnostic which can measure how many particles, $f$, are moving with a particular velocity, $v$. By looking at the distribution of particle velocities, $f(v)$, we can learn something about the bulk properties of the plasma. In collisional plasmas we expect this distribution function, $f$, to be well described by a Maxwellian:

$$
f(v) = A \exp{\left[-\frac{v^2}{v_{th}^2}\right]}
$$

where $A$ is some constant (we'll come back to this later) and $v_{th}$ is the thermal velocity given by:

$$
 v_{th} = \sqrt{\frac{2 k_B T}{m}}
$$

where $k_B$ is the Boltzmann constant, $T$ is the temperature and $m$ is the particle mass.

Given $f(v)$ we want to determine the temperature of the plasma. Here we will see how to do this with python.

## Getting the measurements

We've applied our diagnostic to a hydrogen plasma, measured $f$ for a range of $v$ and wrote these values to file. Now we want to read this data into python so that we can try to determine the temperature. We stored the data into the file `exercise_data/distribution_1.txt` which contains three columns, $v$, $f$ and the error on $f$, $\epsilon$. We can read this in as follows:

In [ ]:
import numpy as np
raw_data = np.loadtxt('exercise_data/distribution_1.txt')
# Check the shape -- see it is 101 x 3. This is 101 measurements of [v, f, error]
print("The shape is ",raw_data.shape)
# Extract the data into sensibly named variables
v_values = raw_data[:, 0] #All 101 values for the first column
f_values = raw_data[:, 1] #All 101 values for the second column
error = raw_data[:, 2] #All 101 values for the third column

## Visualising the measurements

Before we try to make any measurements let us visualise the data

In [ ]:
import matplotlib.pyplot as plt

plt.errorbar(v_values, f_values, fmt = '-x', yerr = error, label = 'Raw data with errors', ecolor = 'red')
plt.xlabel(r'$v$ (m/s)')
plt.ylabel(r'$f(v)$ (#)')
plt.legend(loc = 'best')
plt.grid(True, 'both')
plt.show()

## Measuring the temperature

Now we want to try extracting the temperature from this data. To do this we can try to fit a Maxwellian form to our data:

$$
f(v) = a \exp{\left[-\frac{v^2}{b^2}\right]}
$$

with $a$ and $b$ fitting coefficients. We can then see comparing to our original Maxwellian that $b = v_{th}$ and from the definition of $v_{th}$ we can then rearrange to find $T$:

$$
T = \frac{m b^2}{2 k_B}
$$

In [ ]:
# Get the curve fit function
from scipy.optimize import curve_fit

#Define a function describing the curve we are trying to fit
def maxwellian(v, a, b):
    return a * np.exp(-(v**2)/b**2)

# Initial guess -- the first parameter sets the height of the Maxwellian and the second controls the width
# The y-scale from the plot is ~ 300 and the x scale is ~1e6 so use these as guesses here.
initial_guess = [300.0, 1e6]

fit_params, fit_covariance = curve_fit(maxwellian, v_values, f_values, p0 = [300, 1e6])

print('The fitted value of a is :',fit_params[0])
print('The fitted value of b is :',fit_params[1])

# Always good to visually check the quality of the fit so let us plot is
plt.errorbar(v_values, f_values, fmt = '-x', yerr = error, label = 'Raw data with errors', ecolor = 'red')
plt.plot(v_values, maxwellian(v_values, fit_params[0], fit_params[1]), label = 'Fitted function')
plt.xlabel(r'$v$ (m/s)')
plt.ylabel(r'$f(v)$ (#)')
plt.legend(loc = 'best')
plt.grid(True, 'both')
plt.show()

# Now we can calculate the temperature from the value of b. We need the proton mass and Boltzmann
# constant to do this. Here we get these from the scipy constants module:
from scipy.constants import proton_mass, Boltzmann

temperature = fit_params[1]**2 * proton_mass / (2 * Boltzmann)

print('The temperature deduced from the fit is',temperature,' Kelvin.')

## Summary

Here we have seen how we can read in data, inspect it, fit a model to it and use this to extract some meaningful values in a few lines of python code. This helps to highlight some of the power which comes from the use of appropriate modules in python. You will likely find that much of the time you can essentially "glue together" module methods to produce a tool that does what you want.

However, we could go further in this analysis. Our quoted temperature doesn't include any estimate of the error, how can we get this? What about the error on the measurements, we've only used this in the plotting but should we also use this in the fit?


### Tasks

1. Read the [`curve_fit` documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) to see if you can work out how to modify the example code to include errors on $f$ in the fit _and_ how to calculate the error on the resulting fit parameter `b`. _Hint: a web search containing "curve_fit error on..." might also help._
2. Modify the code to at least calculate the error on `b` and propagate this through to give you an error on $T$. The data was generated with $T = 10,000,000 K$, does the value from the fit agree within error?
3. The fit parameter, `a`, corresponds to the Maxwellian parameter $A$. In fact, here we can define $A = 4 n/\left(\sqrt{\pi} v_{th}\right)$ where $n$ is known as the number density of the plasma. Given the existing code work out the number density of the plasma (ideally including errors).
4. The number density, $n$, can also be written as $n = \int_{-\infty}^\infty{f(v) dv}$. Given the $f(v)$ data from experiment calculate $n$ without fitting. Compare your value to the fit from 3, do they agree?